## Using surprise 
- Read data
- Build basic user or item based models
- Grid Search 
- Top N users or items

https://anaconda.org/conda-forge/scikit-surprise

In [1]:
import pandas as pd
import surprise

In [2]:
## To read a file using surprise, one needs to make sure that data is in a specific format, there are two common ways to read a dataset, to be used in the library
# Reading from dataframe
# Reading from a text file

# Reading data from a dataframe
df=pd.read_csv("sample_data.csv")
df.head()

user  rating  item
0     1       2     1
1     2       2     1
2     3       3     2
3     4       3     2
4     5       1     1

In [3]:
# Surprise expects data to have three columns, user, rating and item. The spelling of these columns names should be as given. If your dataframe has other columns or column names are differen, remember to change them before trying to reading them in surprise

# We will need to create a reader object before we can load our dataframe into surprise 
reader=surprise.dataset.Reader(line_format='user rating item',rating_scale=(1,5))

In [4]:
data=surprise.dataset.Dataset.load_from_df(df,reader=reader)

In [5]:
data.raw_ratings

[(1, 2, 1.0, None),
 (2, 2, 1.0, None),
 (3, 3, 2.0, None),
 (4, 3, 2.0, None),
 (5, 1, 1.0, None)]

In [6]:
# We can load the dataset from a text file as well, directly, just make sure the text file 
# has three columns named as user, rating and item
reader=surprise.dataset.Reader(line_format='user rating item',sep=",", 
                               rating_scale=(1,5),skip_lines=1)

In [7]:
data1=surprise.dataset.Dataset.load_from_file("sample_data.csv",reader=reader)

In [8]:
data1.raw_ratings

[('1', '1', 2.0, None),
 ('2', '1', 2.0, None),
 ('3', '2', 3.0, None),
 ('4', '2', 3.0, None),
 ('5', '1', 1.0, None)]

##### Let's now, work with a slightly larger dataset and train memory based collaborative filtering models

In [9]:
mr=pd.read_csv("ratings.csv")
print(mr.head())
mr.drop('timestamp',axis=1,inplace=True)
mr.rename(columns={'userId':'user','movieId':'item','rating':'rating'},inplace=True)

   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205


In [10]:
# user, item, rating on scale of 1 to 5
reader=surprise.dataset.Reader(line_format='user item rating', rating_scale=(1,5))

In [11]:
mr_train=surprise.dataset.Dataset.load_from_df(mr,reader=reader)
mr_trainset=mr_train.build_full_trainset()

In [12]:
## Create a neighbourhood based user and item based collaborative filtering model
import surprise.prediction_algorithms.knns as knns
knnbasic=knns.KNNBasic(k=40,min_k=1,sims_options={'name':'cosine','user_based':True})

In [13]:
knnbasic.fit(mr_trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [14]:
mr.head()

user  item  rating
0     1    31     2.5
1     1  1029     3.0
2     1  1061     3.0
3     1  1129     2.0
4     1  1172     4.0

In [15]:
knnbasic.predict(uid=1,iid=31,r_ui=2.5)

Prediction(uid=1, iid=31, r_ui=2.5, est=2.986320319817485, details={'actual_k': 40, 'was_impossible': False})

In [16]:
## Lets build an item based collaborative filter
knnbasic=knns.KNNBasic(k=40,min_k=1,sims_options={'name':'cosine','user_based':False})

In [17]:
knnbasic.fit(mr_trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [18]:
knnbasic.predict(uid=1,iid=31)

Prediction(uid=1, iid=31, r_ui=None, est=2.986320319817485, details={'actual_k': 40, 'was_impossible': False})

In [19]:
## Collaborative filter with average effects
knnbasic=knns.KNNWithMeans(k=40,min_k=1,sims_options={'name':'pearson','user_based':False})
knnbasic.fit(mr_trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [20]:
knnbasic.predict(uid=1,iid=31)

Prediction(uid=1, iid=31, r_ui=None, est=2.1033560498894315, details={'actual_k': 40, 'was_impossible': False})

In [23]:
## Instead of  using just one train set, we can split the data into parts
# and then evaluate the model performance out of sample
mr_train.split(n_folds=3)
surprise.model_selection.cross_validate(knns.KNNBasic(k=40,sims_options={'name':'cosine','user_based':False}),mr_train,cv = 5 ,verbose = True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9690  0.9642  0.9675  0.9729  0.9616  0.9670  0.0039  
MAE (testset)     0.7449  0.7413  0.7429  0.7510  0.7376  0.7435  0.0044  
Fit time          0.32    0.37    0.39    0.35    0.37    0.36    0.02    
Test time         2.65    2.09    2.54    2.61    2.53    2.48    0.20    


{'test_rmse': array([0.96897575, 0.96420698, 0.96753409, 0.97291633, 0.96156343]),
 'test_mae': array([0.74493046, 0.74134237, 0.7428643 , 0.75095977, 0.73760194]),
 'fit_time': (0.31959080696105957,
  0.37410688400268555,
  0.38898324966430664,
  0.3470320701599121,
  0.3718276023864746),
 'test_time': (2.652984380722046,
  2.0912933349609375,
  2.5368947982788086,
  2.605311870574951,
  2.5314154624938965)}

In [24]:
## Build a collaborative filter model with average effects
surprise.model_selection.cross_validate(knns.KNNWithMeans(k=40,sims_options={'name':'cosine','user_based':False}),mr_train,cv = 5 ,verbose = True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9213  0.9180  0.9128  0.9203  0.9179  0.9181  0.0030  
MAE (testset)     0.7019  0.7035  0.6996  0.7051  0.7041  0.7028  0.0019  
Fit time          0.35    0.37    0.38    0.43    0.44    0.39    0.03    
Test time         2.65    2.48    2.78    3.03    3.05    2.80    0.22    


{'test_rmse': array([0.92131994, 0.91803735, 0.91277812, 0.92034528, 0.9179295 ]),
 'test_mae': array([0.70191732, 0.70351285, 0.69958273, 0.70508944, 0.70408149]),
 'fit_time': (0.35298728942871094,
  0.3685038089752197,
  0.384105920791626,
  0.4288177490234375,
  0.43738818168640137),
 'test_time': (2.6524956226348877,
  2.479090690612793,
  2.7798171043395996,
  3.027111291885376,
  3.0524649620056152)}

In [25]:
## Doing Grid Search
param_grid = {'k': [10, 20],
              'sim_options': {'name': ['msd', 'cosine'],
                              'user_based': [False]}
              }

In [26]:
algo=knns.KNNWithMeans

In [32]:
grid_search = surprise.model_selection.search.GridSearchCV(algo,param_grid=param_grid, measures=['RMSE', 'MAE'])

In [36]:
grid_search.fit(mr_train)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix.

In [ ]:
grid_search.

In [59]:
print(grid_search.best_params['rmse'])
print(grid_search.best_params['mae'])

{'k': 20, 'sim_options': {'name': 'msd', 'user_based': False}}
{'k': 20, 'sim_options': {'name': 'msd', 'user_based': False}}


In [60]:
print(grid_search.best_score['rmse'])
print(grid_search.best_score['mae'])

0.9232157259583346
0.7081695260165326


In [62]:
## Top 5 recommendations for an item
model=knns.KNNWithMeans(k=20,sim_options={'name': 'msd', 'user_based': False})
model.fit(mr_trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [63]:
mr.head()

user  item  rating
0     1    31     2.5
1     1  1029     3.0
2     1  1061     3.0
3     1  1129     2.0
4     1  1172     4.0

In [64]:
mr_trainset.to_inner_iid(1061)

2

In [65]:
model.get_neighbors(mr_trainset.to_inner_iid(1029),5)

[136, 252, 357, 594, 668]

In [66]:
for i in [133, 136, 252, 357, 503]:
   print(mr_trainset.to_raw_iid(i))

27369
50068
2102
4718
26


In [67]:
## Top 5 recommendations for a user
model=knns.KNNWithMeans(k=20,sim_options={'name': 'msd', 'user_based': True})
model.fit(mr_trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [68]:
mr.head()

user  item  rating
0     1    31     2.5
1     1  1029     3.0
2     1  1061     3.0
3     1  1129     2.0
4     1  1172     4.0

In [69]:
mr_trainset.to_inner_uid(1)
model.get_neighbors(mr_trainset.to_inner_uid(1),5)

[8, 32, 67, 95, 98]

In [70]:
for i in [8, 25, 32, 37, 67]:
    print(mr_trainset.to_raw_uid(i))

9
26
33
38
68
